In [ ]:
#import libarries dan Bikin Arsitektur


%tensorflow_version 1.x
import keras
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import AveragePooling2D
from keras.layers.core import Dense
from keras.layers.core import Activation
from keras.layers.advanced_activations import ELU
from keras.layers.core import Dropout
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import concatenate
from keras.models import Model
from keras.regularizers import l2
from keras import backend as K

class GoogLeNet:
    @staticmethod
    def conv_module(x, K, kX, kY, stride, chanDim,
        padding = "same", reg = 0.0005, name = None):
        # initialize the CONV, BN, and RELU layer names
        (convName, bnName, actName) = (None, None, None)

        # if a layer name was supplied, prepend it
        if name is not None:
            convName = name + "_conv"
            bnName = name + "_bn"
            actName = name + "_act"

        # define a CONV => BN => RELU pattern
        x = Conv2D(K, (kX, kY), strides = stride, padding = padding,
            kernel_regularizer = l2(reg), name = convName)(x)
        x = BatchNormalization(axis = chanDim, name = bnName)(x)
        x = Activation("relu", name = actName)(x)

        # return the block
        return x

    @staticmethod
    def inception_module(x, num1x1, num3x3Reduce, num3x3, num5x5Reduce,
        num5x5, num1x1Proj, chanDim, stage, reg = 0.0005):
        # define the first branch of the Inception module which
        # consists of 1x1 convolutions
        first = GoogLeNet.conv_module(x, num1x1, 1, 1, (1, 1),
            chanDim, reg = reg, name = stage + "_first")

        # define the second branch of the Inception module which
        # consists of 1x1 and 3x3 convolutions
        second = GoogLeNet.conv_module(x, num3x3Reduce, 1, 1, (1, 1),
            chanDim, reg = reg, name = stage + "_second1")
        second = GoogLeNet.conv_module(second, num3x3, 3, 3, (1, 1),
            chanDim, reg = reg, name = stage + "_second2")

        # define the third branch of the Inception module which
        # are both 1x1 and 5x5 convolutions
        third = GoogLeNet.conv_module(x, num5x5Reduce, 1, 1, (1, 1),
            chanDim, reg = reg, name = stage + "_third1")
        third = GoogLeNet.conv_module(third, num5x5, 5, 5, (1, 1),
            chanDim, reg = reg, name = stage + "_third2")

        # define the fourth branch of the Inception module which
        # is the POOL projection
        fourth = MaxPooling2D((3, 3), strides = (1, 1), padding = "same",
            name = stage + "_pool")(x)
        fourth = GoogLeNet.conv_module(fourth, num1x1Proj, 1, 1, (1, 1),
            chanDim, reg = reg, name = stage + "_fourth")

        # concatenate across the channel dimension
        x = concatenate([first, second, third, fourth], axis = chanDim,
            name = stage + "_mixed")

        # return the block
        return x

    @staticmethod
    def build(width, height, depth, classes, reg = 0.0005):
        # initialize the input shape to be "channel last" and the
        # channels dimension itself
        inputShape = (height, width, depth)
        chanDim = -1

       # if we are using "channel first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

        # define the model input, followed by a sequence of
        # CONV => POOL => (CONV * 2) => POOL layers
        inputs = Input(shape = inputShape)
        x = GoogLeNet.conv_module(inputs, 64, 5, 5, (1, 1),
            chanDim, reg = reg, name = "block1")
        x = MaxPooling2D((3, 3), strides = (2, 2), padding = "same",
            name = "pool1")(x)
        x = GoogLeNet.conv_module(x, 64, 1, 1, (1, 1),
            chanDim, reg = reg, name = "block2")
        x = GoogLeNet.conv_module(x, 192, 3, 3, (1, 1),
            chanDim, reg = reg, name = "block3")
        x = MaxPooling2D((3, 3), strides = (2, 2), padding = "same",
            name = "pool2")(x)

        # apply two Inception module followed by a POOL
        x = GoogLeNet.inception_module(x, 64, 96, 128, 16, 32, 32,
            chanDim, "3a", reg = reg)
        x = GoogLeNet.inception_module(x, 128, 128, 192, 32, 96, 64,
            chanDim, "3b", reg = reg)
        x = MaxPooling2D((3, 3), strides = (2, 2), padding = "same",
            name = "pool3")(x)

        # apply five Inception module followed by POOL
        x = GoogLeNet.inception_module(x, 192, 96, 208, 16, 48, 64,
            chanDim, "4a", reg = reg)
        x = GoogLeNet.inception_module(x, 160, 112, 224, 24, 64, 64,
            chanDim, "4b", reg = reg)
        x = GoogLeNet.inception_module(x, 128, 128, 256, 24, 64, 64,
            chanDim, "4c", reg = reg)
        x = GoogLeNet.inception_module(x, 112, 144, 288, 32, 64, 64,
            chanDim, "4d", reg = reg)
        x = GoogLeNet.inception_module(x, 256, 160, 320, 32, 128, 128,
            chanDim, "4e", reg = reg)
        x = MaxPooling2D((3, 3), strides = (2, 2), padding = "same",
            name = "pool4")(x)

        # apply last two Inception module
        x = GoogLeNet.inception_module(x, 256, 160, 320, 32, 128, 128,
            chanDim, "5a", reg = reg)
        x = GoogLeNet.inception_module(x, 384, 192, 384, 48, 128, 128,
            chanDim, "5b", reg = reg)

        # apply a POOL layer (average) followed by dropout
        x = AveragePooling2D((4, 4), name = "pool5")(x)
        x = Dropout(0.4, name = "do")(x)

        # softmax classifier
        x = Flatten(name = "flatten")(x)
        #x = layers.Dropout(0.5)(x)
        x = Dense(classes, kernel_regularizer = l2(reg), name = "labels")(x)
        x = Activation("softmax", name = "softmax")(x)

        # create the model
        model = Model(inputs, x, name = "googlenet")

        # return the constructed network architecture
        return model

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
#link ke google drive

%tensorflow_version 1.x
import keras
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.optimizers import Adam, SGD

from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image
from imutils import paths
import os



from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
#### Updated ####

#### Updated ####
### The following two lines caused the issue:
import sys
#sys.path.append('/content/gdrive/My Drive/ColabNotebooks')
sys.path.append('/content/gdrive/My Drive')

### Replacing the above with these two lines solved the issue:
import os
#os.chdir('/content/gdrive/My Drive/ColabNotebooks') # Ch working directory to project folder
os.chdir('/content/gdrive/My Drive') # Ch working directory to project folder

###

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#setting path gambar di gdrive

print("[INFO] loading images...")
#imagePaths = paths.list_images('DATA SET TA')
imagePaths = paths.list_images('Dataset')

#imagePaths = paths.list_images('Dataset150')

data = []
labels = []


[INFO] loading images...


In [ ]:
#setting path gambar di gdrive

print("[INFO] loading images...")
imagePathV = paths.list_images('CEK/Validasi')
imagePathD = paths.list_images('CEK/Dataset')

#imagePaths = paths.list_images('Dataset150')

data = []
labelsd = []
val = []
labelsv = []
aug = []
labels2 = []

[INFO] loading images...


In [ ]:
#ambil dataset Asli tanpa augmen warna

for imagePath in imagePaths:

  a = Image.open(imagePath)
  image = np.array(a.resize((224,224))) / 255.0
  data.append(image)

  label = imagePath.split(os.path.sep) [-2]
  labelsd.append(label)

In [ ]:
# #ambil dataset Asli tanpa augmen warna

# for imagePath in imagePathD:

#   a = Image.open(imagePath)
#   image = np.array(a.resize((224,224))) / 255.0
#   data.append(image)

#   label = imagePath.split(os.path.sep) [-2]
#   labelsd.append(label)

In [ ]:
#ambil dataset Asli tanpa augmen warna

for imagePath in imagePathV:

  a = Image.open(imagePath)
  image = np.array(a.resize((224,224))) / 255.0
  val.append(image)

  label = imagePath.split(os.path.sep) [-2]
  labelsv.append(label)

In [ ]:
#Function buat Augmentasi
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
      rotation_range=125,
      width_shift_range=0.7,
      height_shift_range=0.7,
      shear_range=0.7,
      zoom_range=0.7,
      horizontal_flip=True,
      fill_mode='nearest')

In [ ]:
#masukin gambar warna dengan augmentasi dan asli 

for imagePath in imagePathD:

  a = Image.open(imagePath)
  image = np.array(a.resize((224,224))) / 255.0
  x = image


  aug.append(x)
  x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)
  
  i = 0
  label = imagePath.split(os.path.sep) [-2]
  labels2.append(label)
  for batch in datagen.flow(x, batch_size=1):
    mas = x[0,:,:,:] #( 150, 150, 3)
    aug.append(mas)
    label = imagePath.split(os.path.sep) [-2]
    labels2.append(label)
    i += 1
    if i % 3 == 0:
      break 
  


In [ ]:
#masukin gambar warna dengan augmentasi dan asli 

for imagePath in imagePaths:

  a = Image.open(imagePath)
  image = np.array(a.resize((224,224))) / 255.0
  x = image

  data.append(x)
  label = imagePath.split(os.path.sep) [-2]
  labels.append(label)

  aug.append(x)
  x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)
  
  i = 0
  label = imagePath.split(os.path.sep) [-2]
  labels2.append(label)
  for batch in datagen.flow(x, batch_size=1):
    mas = x[0,:,:,:] #( 150, 150, 3)
    aug.append(mas)
    label = imagePath.split(os.path.sep) [-2]
    labels2.append(label)
    i += 1
    if i % 3 == 0:
      break 
  

In [ ]:
#ini one hot encoding dataset tanpa augmentasi, 
  print(labels)
  lb = LabelBinarizer()
  labels = lb.fit_transform(labels)
  #print(labels)

IndentationError: ignored

In [ ]:
#ini one hot encoding dataset tanpa augmentasi, 
print(labelsv)
lb = LabelBinarizer()
labelsv = lb.fit_transform(labelsv)
print(labelsv)

In [ ]:
#ini one hot encoding dataset tanpa augmentasi, 
print(labels2)
lb = LabelBinarizer()
labels2 = lb.fit_transform(labels2)
print(labels2)

In [ ]:
#ini set data Train dan Test kalau pakai dataset asli aja
(trainX,testX, trainY, testY) = train_test_split(np.array(data),
np.array(labels), test_size=0.2)
print(trainX.shape)
print(testX.shape)
#kalau pakai ini yg bawah gak usah di run

In [ ]:
#ini set data Train kalau pake augmen

(trainX,dumbx, trainY, dumby) = train_test_split(np.array(aug),
np.array(labels2),test_size=0.05)
print(trainX.shape)
print(trainY.shape)


In [ ]:
#ini set data Test kalau pake augmen
#ini dan cell atas harus di run

(dumbx,testX, dumby, testY) = train_test_split(np.array(val),
np.array(labelsv), test_size=70)
print(testX.shape)
print(testY.shape)

In [ ]:
#setting arsi dan opt
model = GoogLeNet.build(width = 224, height = 224, depth = 3,
        classes = 4, reg = 0.0002)



In [ ]:
# opt = SGD(lr = 1e-3)
opt = Adam(lr = 1e-3)

model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics = ["accuracy"])

In [ ]:
compile?


In [ ]:
#cek arsi
model.summary()

In [ ]:
#from sklearn.tree import DecisionTreeClassifier
#model = DecisionTreeClassifier()
#running
e = 55
b = 32

H=model.fit(trainX, trainY,validation_data=(testX, testY), batch_size=b, epochs=e, shuffle=True )
predY=model.predict(testX)
print("\n Evaluate the new model against the test set:")
model.evaluate(x=testX, y=testY, batch_size=32)
list_of_metrics_to_plot = ['accuracy']


In [ ]:
#evaluasi
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

In [ ]:
#evaluasi 2
test_score = model.evaluate(testX, testY, batch_size = 32)
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

In [ ]:
H

In [ ]:
#bikin grafik epoch sama Val_acc
from matplotlib import pyplot as plt
import pandas as pd

epochs = H.epoch
hist = pd.DataFrame(H.history)
list_of_metrics_to_plot = ['val_acc']
plot_curve(epochs, hist, list_of_metrics_to_plot)


In [ ]:
#grafik val_acc sama train_acc
import matplotlib.pyplot as plt
f, ax = plt.subplots()
ax.plot([None] + H.history['accuracy'], 'o-')
ax.plot([None] + H.history['val_accuracy'], 'x-')
# Plot legend and use the best location automatically: loc = 0.
ax.legend(['Train acc', 'Validation acc'], loc = 0)
ax.set_title('Training/Validation acc per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('acc')

In [ ]:
#grafik loss train dengan loss val
import matplotlib.pyplot as plt
f, ax = plt.subplots()
ax.plot([None] + H.history['loss'], 'o-')
ax.plot([None] + H.history['val_loss'], 'x-')
# Plot legend and use the best location automatically: loc = 0.
ax.legend(['Train Loss', 'Validation Loss'], loc = 0)
ax.set_title('Training/Validation Loss per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')

In [ ]:
testY

In [ ]:
#Bikin Confusion
from sklearn.metrics import confusion_matrix

labels=["Grade AA", "Grade A", "Grade B","Grade C"]
print(confusion_matrix(testY, predY))


In [ ]:
#run 
from sklearn.metrics import multilabel_confusion_matrix
cm=multilabel_confusion_matrix(testY,predY)
print(cm)


In [ ]:
#run
from sklearn.metrics import multilabel_confusion_matrix

predY[1]
rounded_labels=np.argmax(predY, axis=1)
rounded_labels[1]
rounded_predict=np.argmax(testY, axis=1)
rounded_predict[1]
print(confusion_matrix(rounded_labels, rounded_predict))


In [ ]:
#run
rounded_labels[:,]


In [ ]:
import numpy as np


def plot_confusion_matrix(cm,
                          target_names=['1', '2', '3', '4'],
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    if normalize:
      plt.imshow(cm/7, interpolation='nearest', cmap=cmap)
    else:
      plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.3f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
#run
#gambar confusion matrix
cm = confusion_matrix(rounded_labels,  rounded_predict)
plot_confusion_matrix(cm)

In [ ]:
def conv_module(x, K, kX, kY, stride, chanDim,
        padding = "same", reg = 0.0005, name = None):
        # initialize the CONV, BN, and RELU layer names
        (convName, bnName, actName) = (None, None, None)

        # if a layer name was supplied, prepend it
        if name is not None:
            convName = name + "_conv"
            bnName = name + "_bn"
            actName = name + "_act"

        # define a CONV => BN => RELU pattern
        x = Conv2D(K, (kX, kY), strides = stride, padding = padding,
            kernel_regularizer = l2(reg), name = convName)(x)
        x = BatchNormalization(axis = chanDim, name = bnName)(x)
        x = Activation("relu", name = actName)(x)

        # return the block
        return x

def inception_module(x, num1x1, num3x3Reduce, num3x3, num5x5Reduce,
        num5x5, num1x1Proj, chanDim, stage, reg = 0.0005):
        # define the first branch of the Inception module which
        # consists of 1x1 convolutions
        first = GoogLeNet.conv_module(x, num1x1, 1, 1, (1, 1),
            chanDim, reg = reg, name = stage + "_first")

        # define the second branch of the Inception module which
        # consists of 1x1 and 3x3 convolutions
        second = GoogLeNet.conv_module(x, num3x3Reduce, 1, 1, (1, 1),
            chanDim, reg = reg, name = stage + "_second1")
        second = GoogLeNet.conv_module(second, num3x3, 3, 3, (1, 1),
            chanDim, reg = reg, name = stage + "_second2")

        # define the third branch of the Inception module which
        # are both 1x1 and 5x5 convolutions
        third = GoogLeNet.conv_module(x, num5x5Reduce, 1, 1, (1, 1),
            chanDim, reg = reg, name = stage + "_third1")
        third = GoogLeNet.conv_module(third, num5x5, 5, 5, (1, 1),
            chanDim, reg = reg, name = stage + "_third2")

        # define the fourth branch of the Inception module which
        # is the POOL projection
        fourth = MaxPooling2D((3, 3), strides = (1, 1), padding = "same",
            name = stage + "_pool")(x)
        fourth = GoogLeNet.conv_module(fourth, num1x1Proj, 1, 1, (1, 1),
            chanDim, reg = reg, name = stage + "_fourth")

        # concatenate across the channel dimension
        x = concatenate([first, second, third, fourth], axis = chanDim,
            name = stage + "_mixed")

        # return the block
        return x

width = 224
height = 224
depth = 3
classes = 11
reg = 0.0005
chanDim = -1

        # if we are using "channel first", update the input shape
        # and channels dimension


        # define the model input, followed by a sequence of
        # CONV => POOL => (CONV * 2) => POOL layers
inputs = Input(shape=(224, 224, 3))
x = conv_module(inputs, 64, 5, 5, (1, 1), chanDim, reg = reg, name = "block1")
x = MaxPooling2D((3, 3), strides = (2, 2), padding = "same", name = "pool1")(x)
x = conv_module(x, 64, 1, 1, (1, 1), chanDim, reg = reg, name = "block2")
x = conv_module(x, 192, 3, 3, (1, 1), chanDim, reg = reg, name = "block3")
x = MaxPooling2D((3, 3), strides = (2, 2), padding = "same",name = "pool2")(x)

        # apply two Inception module followed by a POOL
x = inception_module(x, 64, 96, 128, 16, 32, 32, chanDim, "3a", reg = reg)
x = inception_module(x, 128, 128, 192, 32, 96, 64, chanDim, "3b", reg = reg)
x = MaxPooling2D((3, 3), strides = (2, 2), padding = "same", name = "pool3")(x)

        # apply five Inception module followed by POOL
x = inception_module(x, 192, 96, 208, 16, 48, 64, chanDim, "4a", reg = reg)
x = inception_module(x, 160, 112, 224, 24, 64, 64, chanDim, "4b", reg = reg)
x = inception_module(x, 128, 128, 256, 24, 64, 64, chanDim, "4c", reg = reg)
x = inception_module(x, 112, 144, 288, 32, 64, 64, chanDim, "4d", reg = reg)
x = inception_module(x, 256, 160, 320, 32, 128, 128, chanDim, "4e", reg = reg)
x = MaxPooling2D((3, 3), strides = (2, 2), padding = "same", name = "pool4")(x)

        # apply last two Inception module
x = inception_module(x, 256, 160, 320, 32, 128, 128, chanDim, "5a", reg = reg)
x = inception_module(x, 384, 192, 384, 48, 128, 128, chanDim, "5b", reg = reg)

        # apply a POOL layer (average) followed by dropout
x = AveragePooling2D((4, 4), name = "pool5")(x)
x = Dropout(0.4, name = "do")(x)

        # softmax classifier
x = Flatten(name = "flatten")(x)
        #x = layers.Dropout(0.5)(x)
x = Dense(classes, kernel_regularizer = l2(reg), name = "labels")(x)
x = Activation("softmax", name = "softmax")(x)

        # create the model
model = Model(inputs, x, name = "googlenet")


In [ ]:
ls

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
#inputs = Input(shape=(224, 224, 3))
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = Model(inputs, successive_outputs, name = "googlenet")

img = load_img('Dataset/Grade A/IMG_0572.JPG', target_size=(224, 224))

x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# Now let's display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    n_features = feature_map.shape[-1]  # number of features in feature map
    # The feature map has shape (1, size, size, n_features)
    size = feature_map.shape[1]
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      # Postprocess the feature to make it visually palatable
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x
    # Display the grid
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [ ]:
#ini cuma buat ngetes
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img

gambarcek = os.listdir('Dataset/Grade A')
img_path = os.path.join('Dataset/Grade A', gambarcek[2])
img = load_img(img_path, target_size=(224, 224))  # this is a PIL image
x = np.array(img) 
# Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

# The .flow() command below generates batches of randomly transformed images
# It will loop indefinitely, so we need to `break` the loop at some point!
i = 0
for batch in datagen.flow(x, batch_size=1):
  plt.figure(i)
  imgplot = plt.imshow(array_to_img(batch[0]))
  i += 1
  if i % 5 == 0:
    break